In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 2935}


2935

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 5143}


5143

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,9404b61c532d4bdc89aedc650702eb9d,5djeff,2022-03-01T18:17:36.795000Z,1bd9b2a0-0347-4d83-9f99-d80fcbc54eb5,3,Jean,Enemie
1,9404b61c532d4bdc89aedc650702eb9d,5djeff,2022-03-01T18:15:57.470000Z,7e5b5866-e22d-4e36-8db2-8429271224fa,2,Jean,Arnez
2,9404b61c532d4bdc89aedc650702eb9d,5djeff,2022-03-01T18:14:41.393000Z,71eb2156-ced1-43f5-9bbc-f621c5fad69b,2,Thelus,Robenson
3,9404b61c532d4bdc89aedc650702eb9d,5djeff,2022-03-01T18:13:26.891000Z,a8184297-bb60-4b7a-af87-6e20e47c9bb6,4,Otalus,Sonia
4,9404b61c532d4bdc89aedc650702eb9d,5djeff,2022-03-01T18:10:44.143000Z,7757a84f-7905-4085-9e5a-0a060ca99093,1,Jean,Adonise
...,...,...,...,...,...,...,...
2798,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
2799,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
2800,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
2801,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect


,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
5138,9404b61c532d4bdc89aedc650702eb9d,00cfea76-1c6f-4742-bd39-f395f793886d,12,1,non,non,7e5b5866-e22d-4e36-8db2-8429271224fa,child
5139,9404b61c532d4bdc89aedc650702eb9d,b255af66-78c1-4669-9ef1-6b7d7f8ab470,10,2,non,non,7e5b5866-e22d-4e36-8db2-8429271224fa,child
5140,9404b61c532d4bdc89aedc650702eb9d,a4763c52-2cf1-4d9a-b174-fb45eb18b389,10,1,non,non,1bd9b2a0-0347-4d83-9f99-d80fcbc54eb5,child
5141,9404b61c532d4bdc89aedc650702eb9d,f7b64a2b-7e1b-41c4-b837-c9877e5d8cf7,14,2,non,non,1bd9b2a0-0347-4d83-9f99-d80fcbc54eb5,child
